# AI based Natural disaster analysis

### Importing Neccessary Libraries

In [1]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [116]:
tensorflow.__version__

'2.3.0'

In [117]:
tensorflow.keras.__version__

'2.4.0'

### Image Data Agumentation

In [2]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

### Loading our data and performing data agumentation

In [6]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory(r'E:\SB1\dataset\dataset\dataset\test_set',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='categorical')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory(r'E:\SB1\dataset\dataset\dataset\test_set',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='categorical') 

Found 198 images belonging to 4 classes.
Found 198 images belonging to 4 classes.


In [7]:
print(x_train.class_indices)#checking the number of classes

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}


In [8]:
print(x_test.class_indices)#checking the number of classes

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}


In [9]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 64, 1: 29, 2: 61, 3: 44})

### Creating the model

In [16]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and poolingo
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=4, activation='softmax')) # softmax for more than 2



In [17]:
classifier.summary()#summary of our model

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 27, 27, 32)        9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
flatten_2 (Flatten)          (None, 3872)             

### Compiling the model

In [18]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

## Fitting the model

In [19]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=40, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

Epoch 1/40
40/40 [==============================] - 10s 238ms/step - loss: 1.3960 - accuracy: 0.3188 - val_loss: 1.3396 - val_accuracy: 0.3232
Epoch 2/40
40/40 [==============================] - 9s 233ms/step - loss: 1.3957 - accuracy: 0.2444 - val_loss: 1.3194 - val_accuracy: 0.3232
Epoch 3/40
40/40 [==============================] - 9s 235ms/step - loss: 1.3408 - accuracy: 0.4124 - val_loss: 1.2801 - val_accuracy: 0.3232
Epoch 4/40
40/40 [==============================] - 9s 230ms/step - loss: 1.2149 - accuracy: 0.3755 - val_loss: 1.2168 - val_accuracy: 0.4040
Epoch 5/40
40/40 [==============================] - 9s 230ms/step - loss: 1.2063 - accuracy: 0.4209 - val_loss: 1.0724 - val_accuracy: 0.5859
Epoch 6/40
40/40 [==============================] - 9s 233ms/step - loss: 1.0577 - accuracy: 0.5329 - val_loss: 1.1393 - val_accuracy: 0.5404
Epoch 7/40
40/40 [==============================] - 10s 251ms/step - loss: 0.9185 - accuracy: 0.5913 - val_loss: 0.8898 - val_accuracy: 0.6212
Epoc

### Saving our model

In [20]:
# Save the model
classifier.save('disaster_f.h5')

In [21]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)

### Predicting our results

In [61]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
#model = load_model("disaster_f.h5") #loading the model for testing

In [77]:
img = image.load_img(r"E:\SB1\owntest\eq.jpeg",grayscale=False,
                     target_size= (64,64))#loading of the image
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
pred = classifier.predict_classes(x)#predicting the classes
pred

array([2], dtype=int64)

In [78]:
index=['Cyclone','Earthquake','Flood','Wildfire']
result=str(index[pred[0]])
result

'Flood'

In [42]:
!pip install jupyterthemes as jt

  Using cached jupyterthemes-0.20.0-py2.py3-none-any.whl (7.0 MB)


ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as


In [28]:
!jt -t monokai

'jt' is not recognized as an internal or external command,
operable program or batch file.
